In [1]:
ls

CoronaNet/                          README.md
Get_Chinese_Subnational_Data.ipynb  SelfAnalysis.ipynb


In [2]:
ls ../data

OxCGRT_US_states_temp.csv     OxCGRT_latest_responses.csv
OxCGRT_latest.csv             OxCGRT_latest_withnotes.csv
OxCGRT_latest_allchanges.csv  timeseries/


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

In [4]:
pd.set_option('display.max_columns', None)

#### First, we extra original data from `OxCGRT_latest_withnotes`

In [5]:
df_latest_withnotes = pd.read_csv('../data/OxCGRT_latest.csv')
df_latest_withnotes.head(2)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (2,3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,CountryName,CountryCode,RegionName,RegionCode,Date,C1_School closing,C1_Flag,C2_Workplace closing,C2_Flag,C3_Cancel public events,C3_Flag,C4_Restrictions on gatherings,C4_Flag,C5_Close public transport,C5_Flag,C6_Stay at home requirements,C6_Flag,C7_Restrictions on internal movement,C7_Flag,C8_International travel controls,E1_Income support,E1_Flag,E2_Debt/contract relief,E3_Fiscal measures,E4_International support,H1_Public information campaigns,H1_Flag,H2_Testing policy,H3_Contact tracing,H4_Emergency investment in healthcare,H5_Investment in vaccines,M1_Wildcard,ConfirmedCases,ConfirmedDeaths,StringencyIndex,StringencyIndexForDisplay,StringencyLegacyIndex,StringencyLegacyIndexForDisplay,GovernmentResponseIndex,GovernmentResponseIndexForDisplay,ContainmentHealthIndex,ContainmentHealthIndexForDisplay,EconomicSupportIndex,EconomicSupportIndexForDisplay
0,Aruba,ABW,NaN,NaN,20200101,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Aruba,ABW,NaN,NaN,20200102,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Great, so now we need to examine the data from CoronaNet and find the corresponding columns / policy code. 

In [6]:
ls CoronaNet/

CoronaNet_Codebook.pdf         coronanet_release_allvars.csv
README.md                      paper.pdf


In [7]:
df_corona_net = pd.read_csv("./CoronaNet/coronanet_release_allvars.csv", encoding = "ISO-8859-1")
df_corona_net.head()

,Unnamed: 0,record_id,policy_id,entry_type,correct_type,update_type,update_level,description,date_announced,date_start,date_end,country,ISO_A3,ISO_A2,init_country_level,domestic_policy,province,city,type,type_sub_cat,type_text,school_status,target_country,target_geog_level,target_region,target_province,target_city,target_other,target_who_what,target_direction,travel_mechanism,compliance,enforcer,index_high_est,index_med_est,index_low_est,index_country_rank,link,date_updated,recorded_date
0,1,R_1gShvNnZtOkb4X1En,3553952,new_entry,original,NaN,NaN,Bahrain Health Ministry monitoring coronavirus...,2020-01-22,2020-01-22,NaN,Bahrain,BHR,BH,National,0,NaN,NaN,Health Monitoring,NaN,NaN,NaN,All countries,All countries,NaN,NaN,NaN,NaN,All (Travelers + Residents),Inbound/Outbound,All kinds of transport,Voluntary/Recommended but No Penalties,"National Government,Ministry/Department of Health",49.318559,47.307619,45.133747,124.0,https://www.moh.gov.bh/COVID19/Details/3790,2020-04-13,2020-04-13 12:30:56
1,2,R_1pJOFfb6axbgCjeBi,1943262,new_entry,original,NaN,NaN,"Starting from Jan. 22, The Ministry of Health ...",2020-01-22,2020-01-22,NaN,Bahrain,BHR,BH,National,0,NaN,NaN,Public Awareness Measures,Disseminating information related to COVID-19 ...,NaN,NaN,Bahrain,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Voluntary/Recommended but No Penalties,Ministry/Department of Health,49.318559,47.307619,45.133747,124.0,https://web.archive.org/web/20200607212046/htt...,2020-09-25,2020-09-25 17:22:52
2,3,R_3GrLmE8y71iEe17Bi,2408864,new_entry,original,NaN,NaN,MoH of Barbados met in press conference to dis...,2020-01-22,2020-01-22,NaN,Barbados,BRB,BB,National,0,NaN,NaN,Public Awareness Measures,Disseminating information related to COVID-19 ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mandatory (Unspecified/Implied),Ministry/Department of Health,48.812998,46.853783,43.914687,79.0,https://gisbarbados.gov.bb/blog/health-ministr...,2020-04-20,2020-04-20 11:53:09
3,4,R_1Hi5XBHB1zBOJc7En,5424670,new_entry,correction,NaN,NaN,"On January 22, 2020, Canada implements screeni...",2020-01-22,2020-01-22,NaN,Canada,CAN,CA,National,1,NaN,NaN,Health Monitoring,NaN,NaN,NaN,Canada,"One or more countries, but not all countries",NaN,NaN,NaN,NaN,All Residents (Citizen Residents + Foreign Res...,Inbound,Flights,Mandatory (Unspecified/Implied),National Government,49.181419,47.440853,45.250540,135.0,https://www.canada.ca/en/public-health/service...,2020-04-01,2020-04-01 20:17:57
4,5,R_30j9tBDjjkonFQwCx,9584758,new_entry,original,NaN,NaN,"Wuhan, China ordered the public to wear masks ...",2020-01-22,2020-01-22,NaN,China,CHN,CN,Municipal,1,Hubei,Wuhan,Health Resources,Other Health Infrastructure,NaN,NaN,China,A geographical or administrative unit within a...,NaN,Hubei,Wuhan,NaN,All Residents (Citizen Residents + Foreign Res...,Inbound/Outbound,Not Applicable,Mandatory with Legal Penalties (Jail Time),Municipal Government,52.655965,51.598420,50.520158,194.0,http://www.chinanews.com/gn/2020/01-22/9067622...,2020-04-03,2020-04-03 21:42:26


In [38]:
set(df_corona_net.type)

{'Anti-Disinformation Measures',
 'Closure and Regulation of Schools',
 'Curfew',
 'Declaration of Emergency',
 'External Border Restrictions',
 'Health Monitoring',
 'Health Resources',
 'Health Testing',
 'Hygiene',
 'Internal Border Restrictions',
 'Lockdown',
 'New Task Force, Bureau or Administrative Configuration',
 'Other Policy Not Listed Above',
 'Public Awareness Measures',
 'Quarantine',
 'Restriction and Regulation of Businesses',
 'Restriction and Regulation of Government Services',
 'Restrictions of Mass Gatherings',
 'Social Distancing'}

In [8]:
# for now, we only want the chinese data
df_corona_net_china = df_corona_net[df_corona_net.country == 'China']
df_corona_net_china.head(2)

,Unnamed: 0,record_id,policy_id,entry_type,correct_type,update_type,update_level,description,date_announced,date_start,date_end,country,ISO_A3,ISO_A2,init_country_level,domestic_policy,province,city,type,type_sub_cat,type_text,school_status,target_country,target_geog_level,target_region,target_province,target_city,target_other,target_who_what,target_direction,travel_mechanism,compliance,enforcer,index_high_est,index_med_est,index_low_est,index_country_rank,link,date_updated,recorded_date
4,5,R_30j9tBDjjkonFQwCx,9584758,new_entry,original,NaN,NaN,"Wuhan, China ordered the public to wear masks ...",2020-01-22,2020-01-22,NaN,China,CHN,CN,Municipal,1,Hubei,Wuhan,Health Resources,Other Health Infrastructure,NaN,NaN,China,A geographical or administrative unit within a...,NaN,Hubei,Wuhan,NaN,All Residents (Citizen Residents + Foreign Res...,Inbound/Outbound,Not Applicable,Mandatory with Legal Penalties (Jail Time),Municipal Government,52.655965,51.59842,50.520158,194.0,http://www.chinanews.com/gn/2020/01-22/9067622...,2020-04-03,2020-04-03 21:42:26
5,6,R_1OoGDBSpbZQ02oACt,5468900,new_entry,original,NaN,NaN,"Xi'an, China announced that subway stations an...",2020-01-22,2020-01-22,NaN,China,CHN,CN,Municipal,0,Shaanxi,Xi'an,Hygiene,Other Areas Hygiene Measures Applied,NaN,NaN,China,NaN,NaN,Shaanxi,Xi'an,NaN,NaN,NaN,NaN,Mandatory (Unspecified/Implied),Municipal Government,52.655965,51.59842,50.520158,194.0,http://www.sxdaily.com.cn/2020-01/30/content_8...,2020-07-16,2020-07-16 07:39:56


In [9]:
len(df_corona_net_china)

971

In [10]:
set(df_corona_net_china.city.dropna())

{'Beijing',
 'Changzhou',
 'Chengdu',
 "Chengdu, Ya'an, Neijiang, Suining, Guangyuan, Ziyang, Deyang and Mianyang",
 'Chibi',
 'Dongguan',
 'Ezhou',
 'Foshan',
 'Fuzhou',
 'Guangzhou',
 'Guiyang',
 'Hanzhong, Xianyang',
 'Harbin',
 'Huanggang',
 'Huizhou',
 'Jiangmen',
 'Jilin',
 'Jingzhou',
 'Lichuan',
 'Nanjing',
 'Shantou',
 'Shenyang',
 'Shenzhen',
 'Sufenhe',
 'Suifenhe',
 'Suizhou',
 'Tianjin',
 'Wuhan',
 "Xi'an",
 'Xiantao',
 'Xiaogan',
 'Yichang',
 'Zhaoqing',
 'Zheng jiang',
 'Zhengzhou',
 'Zhongshan',
 'Zhuhai'}

In [11]:
len(set(df_corona_net_china.city.dropna()))

37

In [12]:
set(df_corona_net_china.province)

{'Anhui',
 'Beijing',
 'Chongqing',
 'Fujian',
 'Gansu',
 'Guangdong',
 'Guangxi',
 'Guizhou',
 'Hainan',
 'Hebei',
 'Heilongjiang',
 'Henan',
 'Hong Kong',
 'Hubei',
 'Hunan',
 'Inner Mongolia',
 'Jiangsu',
 'Jiangxi',
 'Jilin',
 'Liaoning',
 'Macau',
 'Ningxia Hui Autonomous Region',
 'Qinghai',
 'Shaanxi',
 'Shandong',
 'Shanghai',
 'Shanxi',
 'Sichuan',
 'Tianjin',
 'Tibet',
 'Xinjiang',
 'Yunnan',
 'Zhejiang',
 nan}

In [13]:
len(set(df_corona_net_china.province))

34

In [14]:
# and we only want province-level data
df_corona_net_china = df_corona_net_china[df_corona_net_china.province.notna()]

In [15]:
# Also, it only makes sense to record policies with a clear start and end date for now
df_corona_net_china = df_corona_net_china[(df_corona_net_china.date_end.notna() & 
                                           df_corona_net_china.date_start.notna())]
df_corona_net_china.head(2)

,Unnamed: 0,record_id,policy_id,entry_type,correct_type,update_type,update_level,description,date_announced,date_start,date_end,country,ISO_A3,ISO_A2,init_country_level,domestic_policy,province,city,type,type_sub_cat,type_text,school_status,target_country,target_geog_level,target_region,target_province,target_city,target_other,target_who_what,target_direction,travel_mechanism,compliance,enforcer,index_high_est,index_med_est,index_low_est,index_country_rank,link,date_updated,recorded_date
44,45,R_8vHG9nuSSEa6uJzEn,7376528,new_entry,original,NaN,NaN,China is locking down Wuhan from January 23 10...,2020-01-23,2020-01-23,2020-04-08,China,CHN,CN,Municipal,1,Hubei,Wuhan,Internal Border Restrictions,NaN,NaN,NaN,China,A geographical or administrative unit within a...,NaN,Hubei,Wuhan,NaN,All Residents (Citizen Residents + Foreign Res...,NaN,NaN,Mandatory (Unspecified/Implied),"National Government,Ministry/Department of Hea...",52.942116,51.85504,50.758864,194.0,http://www.gov.cn/xinwen/2020-01/23/content_54...,2020-04-01,2020-04-01 22:56:24
49,50,R_2y1UXXFuCxaGBiEEn,7376528,update,original,End of Policy,NaN,China is locking down Wuhan from January 23 10...,2020-01-23,2020-01-23,2020-04-08,China,CHN,CN,Municipal,0,Hubei,Wuhan,Internal Border Restrictions,NaN,NaN,NaN,China,A geographical or administrative unit within a...,NaN,Hubei,Wuhan,NaN,All Residents (Citizen Residents + Foreign Res...,NaN,NaN,Mandatory (Unspecified/Implied),"National Government,Ministry/Department of Hea...",52.942116,51.85504,50.758864,194.0,http://www.gov.cn/xinwen/2020-01/23/content_54...,2020-09-15,2020-09-15 00:50:01


#### Let's compare the columns and figure out what we need

The meaning of codes can be found [here](https://github.com/OxCGRT/covid-policy-tracker/blob/master/documentation/codebook.md) (oxford) and [here](./CoronaNet/CoronaNet_Codebook.pdf) (coronanet)

In [16]:
df_latest_withnotes.columns

Index(['CountryName', 'CountryCode', 'RegionName', 'RegionCode', 'Date',
       'C1_School closing', 'C1_Flag', 'C2_Workplace closing', 'C2_Flag',
       'C3_Cancel public events', 'C3_Flag', 'C4_Restrictions on gatherings',
       'C4_Flag', 'C5_Close public transport', 'C5_Flag',
       'C6_Stay at home requirements', 'C6_Flag',
       'C7_Restrictions on internal movement', 'C7_Flag',
       'C8_International travel controls', 'E1_Income support', 'E1_Flag',
       'E2_Debt/contract relief', 'E3_Fiscal measures',
       'E4_International support', 'H1_Public information campaigns',
       'H1_Flag', 'H2_Testing policy', 'H3_Contact tracing',
       'H4_Emergency investment in healthcare', 'H5_Investment in vaccines',
       'M1_Wildcard', 'ConfirmedCases', 'ConfirmedDeaths', 'StringencyIndex',
       'StringencyIndexForDisplay', 'StringencyLegacyIndex',
       'StringencyLegacyIndexForDisplay', 'GovernmentResponseIndex',
       'GovernmentResponseIndexForDisplay', 'ContainmentHealth

In [17]:
df_corona_net_china.columns

Index(['Unnamed: 0', 'record_id', 'policy_id', 'entry_type', 'correct_type',
       'update_type', 'update_level', 'description', 'date_announced',
       'date_start', 'date_end', 'country', 'ISO_A3', 'ISO_A2',
       'init_country_level', 'domestic_policy', 'province', 'city', 'type',
       'type_sub_cat', 'type_text', 'school_status', 'target_country',
       'target_geog_level', 'target_region', 'target_province', 'target_city',
       'target_other', 'target_who_what', 'target_direction',
       'travel_mechanism', 'compliance', 'enforcer', 'index_high_est',
       'index_med_est', 'index_low_est', 'index_country_rank', 'link',
       'date_updated', 'recorded_date'],
      dtype='object')

To see how they correspond to each other, check out [README.md](./README.md) in this directory.

For now, we only want province-level data

In [18]:
df = df_corona_net_china[['country', 'ISO_A3', 'province', 'date_start', 'date_end', 'type', 'type_sub_cat', 'target_geog_level', 'travel_mechanism', 'compliance']]
df.head()

,country,ISO_A3,province,date_start,date_end,type,type_sub_cat,target_geog_level,travel_mechanism,compliance
44,China,CHN,Hubei,2020-01-23,2020-04-08,Internal Border Restrictions,NaN,A geographical or administrative unit within a...,NaN,Mandatory (Unspecified/Implied)
49,China,CHN,Hubei,2020-01-23,2020-04-08,Internal Border Restrictions,NaN,A geographical or administrative unit within a...,NaN,Mandatory (Unspecified/Implied)
133,China,CHN,Jiangsu,2020-01-24,2020-03-27,Declaration of Emergency,NaN,NaN,NaN,Mandatory (Unspecified/Implied)
134,China,CHN,Guizhou,2020-01-24,2020-02-24,Declaration of Emergency,NaN,NaN,NaN,Mandatory (Unspecified/Implied)
136,China,CHN,Yunnan,2020-01-24,2020-02-24,Declaration of Emergency,NaN,NaN,NaN,Mandatory (Unspecified/Implied)


In [20]:
province_code = {
 'Anhui': 'AH',
 'Hong Kong': 'HK',
 'Shandong': 'SD',
 'Beijing': 'BJ',
 'Heilongjiang': 'HL',
 'Shanghai': 'SH',
 'Chongqing': 'CQ',
 'Hunan': 'HN',
 'Shaanxi': 'SN',
 'Fujian': 'FJ',
 'Jilin': 'JL',
 'Shanxi': 'SX',
 'Guangdong': 'GD',
 'Jiangsu': 'JS',
 'Tianjin': 'TJ',
 'Gansu': 'GS',
 'Jiangxi': 'JX',
 'Guangxi': 'GX',
 'Liaoning': 'LN',
 'Xinjiang': 'XJ',
 'Guizhou': 'GZ',
 'Macau': 'MO',
 'Tibet': 'XZ',
 'Henan': 'HA',
 'Inner Mongolia': 'NM',
 'Yunnan': 'YN',
 'Hubei': 'HB',
 'Zhejiang': 'ZJ',
 'Hebei': 'HE',
 'Qinghai': 'QH',
 'Hainan': 'HI',
 'Sichuan': 'SC',
 'Ningxia Hui Autonomous Region': 'NX'
}

In [21]:
df_latest_withnotes.columns

Index(['CountryName', 'CountryCode', 'RegionName', 'RegionCode', 'Date',
       'C1_School closing', 'C1_Flag', 'C2_Workplace closing', 'C2_Flag',
       'C3_Cancel public events', 'C3_Flag', 'C4_Restrictions on gatherings',
       'C4_Flag', 'C5_Close public transport', 'C5_Flag',
       'C6_Stay at home requirements', 'C6_Flag',
       'C7_Restrictions on internal movement', 'C7_Flag',
       'C8_International travel controls', 'E1_Income support', 'E1_Flag',
       'E2_Debt/contract relief', 'E3_Fiscal measures',
       'E4_International support', 'H1_Public information campaigns',
       'H1_Flag', 'H2_Testing policy', 'H3_Contact tracing',
       'H4_Emergency investment in healthcare', 'H5_Investment in vaccines',
       'M1_Wildcard', 'ConfirmedCases', 'ConfirmedDeaths', 'StringencyIndex',
       'StringencyIndexForDisplay', 'StringencyLegacyIndex',
       'StringencyLegacyIndexForDisplay', 'GovernmentResponseIndex',
       'GovernmentResponseIndexForDisplay', 'ContainmentHealth

In [22]:
region_code_series = []
for n in df_corona_net_china.province:
    region_code_series.append(province_code[n])
    
region_code_series = pd.Series(region_code_series)
df.insert(3, "region_code", region_code_series)
df.head(3)

,country,ISO_A3,province,region_code,date_start,date_end,type,type_sub_cat,target_geog_level,travel_mechanism,compliance
44,China,CHN,Hubei,HK,2020-01-23,2020-04-08,Internal Border Restrictions,NaN,A geographical or administrative unit within a...,NaN,Mandatory (Unspecified/Implied)
49,China,CHN,Hubei,HK,2020-01-23,2020-04-08,Internal Border Restrictions,NaN,A geographical or administrative unit within a...,NaN,Mandatory (Unspecified/Implied)
133,China,CHN,Jiangsu,HK,2020-01-24,2020-03-27,Declaration of Emergency,NaN,NaN,NaN,Mandatory (Unspecified/Implied)


# C1


## `C1_School closing`

- 0 - no measures
- 1 - recommend closing
- 2 - require closing (only some levels or categories, eg just high school, or just public schools)
- 3 - require closing all levels
- Blank - no data

## `C1_Flag`

- 0 - targeted
- 1 - general
- Blank - no data

In [23]:
C1_School_closing = []
C1_Flag = []

for i in range(len(df)):
    current = df.iloc[i]
    current_type = current.type
    current_compliance = current.compliance
    current_geo_scope = current.target_geog_level
    
    if current_type == "Closure and Regulation of Schools": 
        # Check Voluntary or Mandatory
        if not current_compliance or current_compliance != current_compliance or str(current_compliance) == "":
            C1_School_closing.append(0)
        elif current_compliance[0] == 'V':
            C1_School_closing.append(1)
        else:
            # Note: Currently we don't have college closure policy in the db
            # So if mandatory, it would be partially mandatory
            C1_School_closing.append(2) 
    
    if current_type == "Closure and Regulation of Schools": # TODO: Think of better coding style
        if not current_geo_scope or current_geo_scope != current_geo_scope or len(str(current_geo_scope)) < 3:
            C1_Flag.append(None)
        elif current_geo_scope[:3] == 'All':
            C1_Flag.append(1)
        else:
            C1_Flag.append(0)
        
    else:
        C1_School_closing.append(None)
        C1_Flag.append(None)


assert(len(df) == len(C1_Flag) == len(C1_School_closing))

In [24]:
df.insert(len(df.columns), 'C1_School closing', C1_School_closing)
df.insert(len(df.columns), 'C1_Flag', C1_Flag)

In [25]:
df.head(2)

,country,ISO_A3,province,region_code,date_start,date_end,type,type_sub_cat,target_geog_level,travel_mechanism,compliance,C1_School closing,C1_Flag
44,China,CHN,Hubei,HK,2020-01-23,2020-04-08,Internal Border Restrictions,NaN,A geographical or administrative unit within a...,NaN,Mandatory (Unspecified/Implied),NaN,None
49,China,CHN,Hubei,HK,2020-01-23,2020-04-08,Internal Border Restrictions,NaN,A geographical or administrative unit within a...,NaN,Mandatory (Unspecified/Implied),NaN,None


# C2

## `C2_Workplace closing` - Record closings of workplaces
- 0 - no measures
- 1 - recommend closing (or recommend work from home)
- 2 - require closing (or work from home) for some sectors or categories of workers
- 3 - require closing (or work from home) for all-but-essential workplaces (eg grocery stores, doctors)
- Blank - no data

## `C2_Flag` 
- 0 - targeted
- 1- general
- Blank - no data

In [26]:
set(df[df.type == 'Restriction and Regulation of Businesses'].type_sub_cat)

{'All or unspecified non-essential businesses'}

In [27]:
C2_Workplace_closing = []
C2_Flag = []

for i in range(len(df)):
    current = df.iloc[i]
    current_type = current.type
    current_subtype = current.type_sub_cat
    current_compliance = current.compliance
    current_geo_scope = current.target_geog_level
    
    if current_type == "Restriction and Regulation of Businesses": 
        # Check Voluntary or Mandatory
        if not current_compliance or current_compliance != current_compliance or str(current_compliance) == "":
            C2_Workplace_closing.append(0)
        elif current_compliance[0] == 'V':
            C2_Workplace_closing.append(1)
        else:
            # Mandatory; check for all or partial
            if not current_subtype or current_subtype != current_subtype or len(str(current_subtype)) < 3:
                C2_Workplace_closing.append(2)
            elif current_subtype[:3] == 'All':
                C2_Workplace_closing.append(3)
            else:
                C2_Workplace_closing.append(2)
    
    if current_type == "Restriction and Regulation of Businesses": # TODO: Think of better coding style
        if not current_geo_scope or current_geo_scope != current_geo_scope or len(str(current_geo_scope)) < 3:
            C2_Flag.append(None)
        elif current_geo_scope[:3] == 'All':
            C2_Flag.append(1)
        else:
            C2_Flag.append(0)
        
    else:
        C2_Workplace_closing.append(None)
        C2_Flag.append(None)


assert(len(df) == len(C2_Flag) == len(C2_Workplace_closing))

In [43]:
df.insert(len(df.columns), 'C2_Workplace closing', C2_Workplace_closing)
df.insert(len(df.columns), 'C2_Flag', C2_Flag)

ValueError: cannot insert C2_Flag, already exists

In [29]:
df.iloc[140]

country                                                             China
ISO_A3                                                                CHN
province                                                        Hong Kong
region_code                                                           NaN
date_start                                                     2020-04-01
date_end                                                       2020-04-15
type                             Restriction and Regulation of Businesses
type_sub_cat                  All or unspecified non-essential businesses
target_geog_level            One or more countries, but not all countries
travel_mechanism                                           Not Applicable
compliance              Mandatory with Legal Penalties (Jail Time),Man...
C1_School closing                                                     NaN
C1_Flag                                                              None
C2 Workplace closing                  

In [30]:
df[df["C2_Flag"].notna()]

,country,ISO_A3,province,region_code,date_start,date_end,type,type_sub_cat,target_geog_level,travel_mechanism,compliance,C1_School closing,C1_Flag,C2 Workplace closing,C2_Flag
22885,China,CHN,Hong Kong,NaN,2020-04-01,2020-04-15,Restriction and Regulation of Businesses,All or unspecified non-essential businesses,"One or more countries, but not all countries",Not Applicable,"Mandatory with Legal Penalties (Jail Time),Man...",NaN,None,3.0,0.0


# C3 - Cancellation of *PUBLIC* events

## `C3_Cancel public events` 
- 0 - no measures
- 1 - recommend cancelling
- 2 - require cancelling
- Blank - no data

## `C3_Flag` 
- 0 - targeted
- 1 - general
- Blank - no data

<h1 style="color:red">Note: Skipping C3 for now; waiting for Coronanet to upload updated codebook</h1>

# C4 

## `C4_Restrictions on gatherings` - limits on *PRIVATE* gatherings

- 0 - no restrictions
- 1 - restrictions on very large gatherings (the limit is above 1000 people)
- 2 - restrictions on gatherings between 101-1000 people
- 3 - restrictions on gatherings between 11-100 people
- 4 - restrictions on gatherings of 10 people or less
- Blank - no data

For now, data does not show us group size, so we default to 11-100 people (3) 

## `C4_Flag` - Binary flag for geographic scope

- 0 - targeted
- 1 - general
- Blank - no data

In [41]:
# There SHOULD be more subcategories though, according to the codebook
set(df[df.type == 'Restrictions of Mass Gatherings'].target_geog_level)

{nan}

In [42]:
C4_Restrictions_Gathering = []
C4_Flag = []

for i in range(len(df)):
    current = df.iloc[i]
    current_type = current.type
    # current_subtype = current.type_sub_cat
    # current_compliance = current.compliance
    current_geo_scope = current.target_geog_level
    
    if current_type == "Restrictions of Mass Gatherings": 
        C4_Restrictions_Gathering.append(3)
        
        if not current_geo_scope or current_geo_scope != current_geo_scope or len(str(current_geo_scope)) < 3:
            C4_Flag.append(None)
        elif current_geo_scope[:3] == 'All':
            C4_Flag.append(1)
        else:
            C4_Flag.append(0)
        
    else:
        C4_Restrictions_Gathering.append(None)
        C4_Flag.append(None)


assert(len(df) == len(C4_Flag) == len(C4_Restrictions_Gathering))

In [ ]:
df.insert(len(df.columns), 'C2 Workplace closing', C4_Restrictions_Gathering)
df.insert(len(df.columns), 'C2_Flag', C2_Flag)